Modelos basados en secuencias con Word2Vec
==========================================

Introducción
------------

Los modelos basados en secuencias tienen la fortaleza que toman una secuencia de token (en un determinado orden) y generan una salida dependiendo del tipo de problema que se trate.
 - Seq2Class: Toman una secuencia de tokens y generan una clase
 - Seq2Seq: Toman una secuencia de token y generan otra secuencia de tokens.

Vimos que cuando aplicamos técnicas de Topic Modeling, intentamos reducir la cantidad de dimensiones de nuestras representaciones de palabras para luego utilizar un clasificador para resolver la tarea en cuestión. Sin embargo, la suponsición básica de ese tipo de modelos es que un texto no es mas que una distribución de palabras (bag of words). Sin embargo, nosotros sabemos que un texto es una secuencia de palabras donde importa el orden. Para capturar este tipo de propiedades podemos utilizar modelos basados en secuencias.

### Para ejecutar este notebook

Para ejecutar este notebook, instale las siguientes librerias:

In [12]:
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Datasets/mascorpus/tweets_marketing.csv \
    --quiet --no-clobber --directory-prefix ./Datasets/mascorpus/
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/m72109/nlp/normalization.py \
    --quiet --no-clobber --directory-prefix ./m72109/nlp/
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/m72109/nlp/transformation.py \
    --quiet --no-clobber --directory-prefix ./m72109/nlp/

!wget https://raw.githubusercontent.com/santiagxf/M72109/master/docs/nlp/neural/sequences-word2vec.txt \
    --quiet --no-clobber
!pip install -r sequences-word2vec.txt --quiet

Descargamos nuestros vectores de word2vec en español

In [2]:
!mkdir -p ./Models/Word2Vec
!wget https://santiagxf.blob.core.windows.net/public/Word2Vec/model-es.bin \
    --quiet --no-clobber

In [2]:
import warnings
warnings.filterwarnings('ignore')

Instalamos las librerias necesarias

In [4]:
!python -m spacy download es_core_news_sm

2023-09-17 23:01:25.369405: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-17 23:01:27.459300: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 27.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


Cargamos el set de datos

In [1]:
import pandas as pd

tweets = pd.read_csv('Datasets/mascorpus/tweets_marketing.csv')

## Preprocesamiento de texto

Al igual que con Topic Modeling, nuestro primer paso es preprocesar el texto. Para focalizarnos en Word2Vec en este modulo, les preparé un modulo TweetTextNormalizer que hará todo el preprocesamiento por nosotros. Pueden explorar los parametros que recibe el constructor de esta clase para ver que opciones podemos configurar como Stemmer, Lemmatization, etc.

En lo particular, estamos creando un TweetTextNormalizer que:
 - Aplicará un tokenizer especifico para Twitter
 - Eliminará stop words
 - Aplicará lemmatization
 - Eliminará URLs
 - Eliminará acentos
 - Eliminará las mayusculas

Adicionalmente, el parametro text_to_sequence=True indica que la salida de este proceso no serán oraciones sino que tokens.

In [2]:
from m72109.nlp.normalization import TweetTextNormalizer

In [3]:
normalizer = TweetTextNormalizer(preserve_case=False, return_tokens=True)

Transformemos el texto:

In [4]:
tweets_text = normalizer.transform(tweets['TEXTO'])

100%|██████████| 3763/3763 [04:13<00:00, 14.86it/s]


## Vectorización de las palabras

En las actividades anteriores utilizamos siempre un TF-IDF vectorizer para generar los vectores. En esta oportunidad utilizaremos Word2Vec utilizando un modelo pre-entrenado para el idioma español. Adicionalmente, vemos que este vectorizer tiene el parametro sequence_to_idx en Verdadero. Esto significa que no queremos que como salida obtengamos los vectores de Word2Vec, sino que queremos "el indice" que se corresponde a la palabra en una matriz de indice-palabra/vectores.

In [5]:
from m72109.nlp.transformation import Word2VecVectorizer

In [6]:
w2v = Word2VecVectorizer(model='/content/model-es.bin', sequence_to_idx=True)

/usr/local/lib/python3.10/dist-packages/gensim/models/keyedvectors.py:551: UserWarning: Adding single vectors to a KeyedVectors which grows by one each time can be costly. Consider adding in batches or preallocating to the required size.
  warnings.warn(
100%|██████████| 2656058/2656058 [00:09<00:00, 267102.64it/s]


In [7]:
tweets_text = w2v.transform(tweets_text)

100%|██████████| 3763/3763 [00:00<00:00, 19669.41it/s]


## Construirmos un modelo basado en secuencias

### Ajustando la longitud de las secuencias

Los modelos basados en secuencias pueden adaptarse a cualquier longitud de secuencia, sin embargo, los parametros de nuestras redes neuronales deberan ser fijos. Para esto definiermos una longitud máxima de la secuencia que vamos analizar. Para esto podemos utilizar un valor especifico o utilizar el valor máximo de tokens que hay en nuestro corpus.

La siguiente clase PadSequenceTransformer es un modulo que les preparé para simplificar este procesamiento. El mismo se encarga de ajustar cualquier secuencia para que tenga exactamente max_seq_len. Cuando la lingitud es mejor, se completan con ceros.

In [18]:
from m72109.nlp.transformation import PadSequenceTransformer

In [19]:
max_seq_len = 100

In [26]:
tweets_text = seq2seq.transform(tweets_text)

### Construyendo el modelo

Para construir nuestro modelo, utilizaremos TensorFlow. En particular utilizaremos la API de Keras que nos permite componer modelos de redes neuronales como una secuencia de pasos o capas que se conectan en una dirección.

Utilizemos los siguientes tipos de capas:

 *  **Embedding:** Esta capa transforma vectores que representan indices dentro de una matriz en representaciones vectoriales densas. Básicamente en este caso nos resolverá la busqueda de las representaciones vectoriales para nuestras palabras.
 * **SpatialDropout1D:** Este tipo de capas ayudan a promover la independencia entre filtros (feature maps). Funciona en forma analoga a Dropout pero en lugar de desconectar elementos individuales, desconecta el filtro completo.
 * **LSTM:** Long Short-Term Memory layer - Hochreiter 1997
 * **Dense:** Una típica capa de una red neuronal completamente conectada (fully connected)

Algunos detalles para notar:

 * `loss='sparse_categorical_crossentropy'`, este problema de clasificación (crossentropy) de más de una clase (categorical). Sin embargo, nuestro output produce probabilidades de cada una de las clases posibles (7) en forma one-hot encoding.
 * `metrics=['accuracy']`: Si bien nuestra metrica es accuracy, Keras hará un promedio ponderado del accuracy de cada clase. Este es el comportamiento por defecto.

In [51]:
embedding_weights = w2v.get_weights()

100%|██████████| 2656058/2656058 [00:10<00:00, 241610.78it/s]


> El método `get_weights()` construye la matríz de indice-palabra/vector que luego será utilizado para encontrar los vectores correspondientes de cada palabra. Esta matriz tiene dimensiones m x n, donde m es la cantidad de palabras del vocabulario y n la dimensión de los vectores de word2vec. En este caso trabajamos con vectores de dimensionalidad 100.

In [27]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input, SpatialDropout1D

In [28]:
model = Sequential([
    Embedding(w2v.vocab_size, w2v.emdedding_size,
              weights=[embedding_weights],
              trainable=False,
              mask_zero=True),
    SpatialDropout1D(0.2),
    LSTM(w2v.emdedding_size),
    Dense(7, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Podemos inspeccionar el modelo:

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         265605800 
                                                                 
 spatial_dropout1d (Spatial  (None, None, 100)         0         
 Dropout1D)                                                      
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 7)                 707       
                                                                 
Total params: 265686907 (1013.52 MB)
Trainable params: 81107 (316.82 KB)
Non-trainable params: 265605800 (1013.21 MB)
_________________________________________________________________


Antes de continuar, separemos nuestro conjunto de datos en entrenamiento y testing y codifiquemos la variable a predecir:

In [32]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
tweets_sector = encoder.fit_transform(tweets['SECTOR'])

In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tweets_text, tweets_sector,
                                                    test_size=0.33,
                                                    stratify=tweets_sector)

Entrenamos nuestro modelo:

In [34]:
history = model.fit(X_train, y_train, epochs=50)

Epoch 1/50
79/79 [==============================] - 11s 107ms/step - loss: 1.3717 - accuracy: 0.5256
Epoch 2/50
79/79 [==============================] - 9s 109ms/step - loss: 0.7200 - accuracy: 0.7949
Epoch 3/50
79/79 [==============================] - 9s 108ms/step - loss: 0.5580 - accuracy: 0.8429
Epoch 4/50
79/79 [==============================] - 9s 116ms/step - loss: 0.4785 - accuracy: 0.8643
Epoch 5/50
79/79 [==============================] - 9s 114ms/step - loss: 0.4193 - accuracy: 0.8782
Epoch 6/50
79/79 [==============================] - 9s 113ms/step - loss: 0.3813 - accuracy: 0.8862
Epoch 7/50
79/79 [==============================] - 9s 117ms/step - loss: 0.3569 - accuracy: 0.8921
Epoch 8/50
79/79 [==============================] - 9s 111ms/step - loss: 0.3190 - accuracy: 0.8981
Epoch 9/50
79/79 [==============================] - 11s 143ms/step - loss: 0.3297 - accuracy: 0.8933
Epoch 10/50
79/79 [==============================] - 9s 109ms/step - loss: 0.2837 - accuracy: 0.90

## Evalución de los resultados

Probamos su performance utilizando el test set

In [42]:
predictions = model.predict(X_test)

39/39 [==============================] - 2s 44ms/step


In [43]:
import numpy as np

predictions = np.argmax(predictions, axis=1)

Veamos el reporte:

In [50]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions, target_names=encoder.classes_))

              precision    recall  f1-score   support

ALIMENTACION       0.99      0.90      0.94       110
  AUTOMOCION       0.84      0.94      0.89       148
       BANCA       0.95      0.90      0.93       198
     BEBIDAS       0.91      0.91      0.91       223
    DEPORTES       0.95      0.95      0.95       216
      RETAIL       0.89      0.92      0.91       268
       TELCO       0.94      0.86      0.90        79

    accuracy                           0.92      1242
   macro avg       0.93      0.91      0.92      1242
weighted avg       0.92      0.92      0.92      1242

